## Instagram crawler sample

In [297]:
from selenium import webdriver
import urllib.parse
from urllib.request import Request, urlopen
import time
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from urllib.error import HTTPError

In [298]:
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [299]:
# 피드 링크 가져오기

def get_feeds(driver):
    feed_list = driver.find_elements_by_tag_name("div")
    feed_links = []
    for feed in feed_list:
        class_name = feed.get_attribute("class")
        if class_name.startswith("v1Nh3"):
            link = feed.find_element_by_tag_name("a").get_attribute("href")
            feed_links.append(link)
    return feed_links

In [353]:
def get_hashtags(driver):
    tags = []
    data = driver.find_elements_by_tag_name("a")
    for datum in data:
        href_val = datum.get_attribute("href")
        if "/explore/tags/" in href_val:
            tags.append(datum.text.replace("#",""))

    return tags

In [301]:
def get_taggedaccs(driver):
    acc_list=[]
    upload_accs = driver.find_elements_by_tag_name("a")
    try:
        for acc in upload_accs:
            class_name = acc.get_attribute("class")
            if class_name == "notranslate":
                acc_list.append(acc.text.replace("@",""))
    except:
        pass
    return acc_list

In [354]:
# 피드 내용 가져오기

def get_feed_content(driver, info):
    # post content
    post_content=driver.find_element_by_class_name("C4VMK")
    info["post_content"] = post_content.text
    
    # hashtags and taagged accounts
    hashtags = get_hashtags(driver)
    tagaccs = get_taggedaccs(driver)
    info["hashtags"] = hashtags
    info["taggedaccs"] = tagaccs

    
    # post likes
    try:
        likes = driver.find_element_by_class_name("Nm9Fw")
        info["likes"] = likes.find_element_by_tag_name("span").text
        info["views"] = "unknown"
    except:
        # 좋아요 외 영상 조회수 확인
        views = driver.find_element_by_class_name("vcOH2")
        info["views"] = views.find_element_by_tag_name("span").text
        info["likes"] = "unknown"
    
    # post date
    date = driver.find_element_by_tag_name("time")
    info["date"] = date.get_attribute("datetime")
        
    return info["date"]

In [303]:
chromedriver="./chromedriver_win32/chromedriver.exe"
driver=webdriver.Chrome(chromedriver)

In [304]:
## Selenium 속도 향상
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 
                                                    'images': 2, 
                                                    'plugins' : 2, 
                                                    'popups': 2, 
                                                    'geolocation': 2, 
                                                    'notifications' : 2, 
                                                    'auto_select_certificate': 2, 
                                                    'fullscreen' : 2, 
                                                    'mouselock' : 2, 
                                                    'mixed_script': 2,
                                                    'media_stream' : 2, 
                                                    'media_stream_mic' : 2, 
                                                    'media_stream_camera': 2, 
                                                    'protocol_handlers' : 2, 
                                                    'ppapi_broker' : 2, 
                                                    'automatic_downloads': 2, 
                                                    'midi_sysex' : 2, 
                                                    'push_messaging' : 2, 
                                                    'ssl_cert_decisions': 2, 
                                                    'metro_switch_to_desktop' : 2, 
                                                    'protected_media_identifier': 2, 
                                                    'app_banner': 2, 
                                                    'site_engagement' : 2, 
                                                    'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

In [305]:
# login
loginUrl = 'https://www.instagram.com/accounts/login/'
driver.get(loginUrl)
time.sleep(2)

# username = input("유저 아이디 입력: ")
username ="soft0gram"
userpw="Yunastellar"

driver.find_element_by_name('username').send_keys(username)
driver.find_element_by_name('password').send_keys(userpw)
time.sleep(2)
driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF').click()
time.sleep(20)

In [306]:
# 정보 나중에 저장하기 클릭하고 넘어가기
try:
    driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
    time.sleep(3)
except:
    pass
# 설정 나중에하기 클릭하고 넘어가기
try:
    driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
    time.sleep(3)
except:
    pass

In [307]:
#keyword=input("해시태그를 입력하세요: ")
keyword="골프"
encText = urllib.parse.quote(keyword)

In [308]:
url="https://www.instagram.com/explore/tags/"+str(encText)+'/'
driver.get(url)
time.sleep(3)

In [309]:
# scroll_time = int(input("스크롤 시간 입력: "))
doScrollDown(10)

In [310]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [311]:
feed_links = get_feeds(driver)
print(len(feed_links))

42


In [315]:
all_accounts = set()
all_posts_info = {}
post_index = 0
for link in feed_links:
    post_link = link
    driver.get(str(post_link))
    time.sleep(3)
    info = {}
    info["link"] = post_link
    
    # account name
    account=driver.find_elements_by_tag_name("a")
    for acc in account:
        if acc.get_attribute("class").startswith("sqdOP"):
            info["account"] = acc.text
            break

    all_accounts.add(info["account"])

    get_feed_content(driver, info)
    
    all_posts_info[post_index] = info
    post_index +=1
    print(info)
    
    break

{'link': 'https://www.instagram.com/p/CSJ36ialJ59/', 'account': 'soyoung_pro', 'post_content': 'soyoung_pro\n⠀\n유띨리띠 🏌🏻\u200d♀️\n⠀\n@mizuno_golfapparel_kr\n1일', 'hashtags': [], 'taggedaccs': ['mizuno_golfapparel_kr'], 'likes': '423', 'views': 'unknown', 'date': '2021-08-04T13:32:19.000Z'}


In [355]:
# 정보에 있는 계정 수집

all_acc_info = []
for acc in all_accounts:
    info = {}

    link = "https://instagram.com/"+acc+"/"
     #account info
    #try: 
        #html = urlopen(link)
    #except HTTPError as e:
        #print(e)
    if link == "":
        pass
    else:
        driver.get(link)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(3)

        acc_info = driver.find_elements_by_class_name("g47SY ")
        acc_info_list = []
        for num in acc_info:
            acc_info_list.append(num)
        info["account"] = acc
        info["account_posts"] = acc_info_list[0].text
        info["account_followers"] = acc_info_list[1].text
        info["account_following"] = acc_info_list[2].text
        info["account_intro"] = driver.find_element_by_class_name("-vDIg").text

        feed_links = get_feeds(driver)
        account_feeds = []
        for feed in feed_links:
            driver.get(feed)
            time.sleep(2)

            feed_content = {}
            try:
                date = get_feed_content(driver, feed_content)
                account_feeds.append(feed_content)
            except:
                driver.get(feed)
                time.sleep(2)
                
                date = get_feed_content(driver, feed_content)
                account_feeds.append(feed_content)

        info["account_feeds"] = account_feeds
        all_acc_info.append(info)
        print(all_acc_info)
    
    break

[{'account': 'soyoung_pro', 'account_posts': '593', 'account_followers': '20천', 'account_following': '168', 'account_intro': '안소영프로\n골프 강사\n🙋🏻\u200d♀️\n⛳️ KLPGA 507/ TPI Level 1\n✏️ 고대 체육학 석사 / 건대 골프지도학과\n💎 위드골프\n🎬 유튜브_안소영프로의 About Golf\nyoutu.be/ohXMEKgFPt0', 'account_feeds': [{'post_content': 'soyoung_pro\n⠀\n회원님이 한땀 한땀 만들어주신 귀여운 목걸이❣️\n얼음된 몽끌이.. 움직여라 오바🕹\n⠀\n5시간', 'hashtags': ['토이푸들', '푸들', '멍스타그램', '인스타독', '애프리푸들', '몽끌이', '강아지목걸이', 'doggy', 'poodle', 'instadog'], 'taggedaccs': [], 'likes': '112', 'views': 'unknown', 'date': '2021-08-05T13:02:48.000Z'}, {'post_content': 'soyoung_pro\n⠀\n40m 어프로치🏌🏻\u200d♀️\n⠀\n@mizuno_golfapparel_kr\n5시간', 'hashtags': ['골스타그램', '골프', '골프레슨', '트랙맨골프', '스윙교정', '골프스윙', '인스타골프', '미즈노골프어패럴', '어프로치', '안소영프로', 'golflessons', 'instagolf', 'golfinstructor', 'golfpractice', 'trackmangolf', 'golf', 'golfswing', 'klpga'], 'taggedaccs': ['mizuno_golfapparel_kr'], 'likes': '193', 'views': 'unknown', 'date': '2021-08-05T12:33:32.000Z'}, {'post_content': 'soy

In [356]:
# 마지막에 엑셀로 정리하기

df = pd.json_normalize(all_acc_info, record_path=["account_feeds"],
                      meta=['account', 
                            "account_posts", 
                            "account_followers",
                            "account_following", 
                            "account_intro"])
df = df.reindex(columns=['account', 
                         'account_posts',
                         'account_followers',
                         "account_following",
                         "account_intro",
                         "post_content",
                         "hashtags",
                         "taggedaccs",
                         "likes",
                         "views",
                         "date"])
df.head(5)

,account,account_posts,account_followers,account_following,account_intro,post_content,hashtags,taggedaccs,likes,views,date
0,soyoung_pro,593,20천,168,안소영프로\n골프 강사\n🙋🏻‍♀️\n⛳️ KLPGA 507/ TPI ...,soyoung_pro\n⠀\n회원님이 한땀 한땀 만들어주신 귀여운 목걸이❣️\n얼음...,"[토이푸들, 푸들, 멍스타그램, 인스타독, 애프리푸들, 몽끌이, 강아지목걸이, do...",[],112,unknown,2021-08-05T13:02:48.000Z
1,soyoung_pro,593,20천,168,안소영프로\n골프 강사\n🙋🏻‍♀️\n⛳️ KLPGA 507/ TPI ...,soyoung_pro\n⠀\n40m 어프로치🏌🏻‍♀️\n⠀\n@mizuno_golf...,"[골스타그램, 골프, 골프레슨, 트랙맨골프, 스윙교정, 골프스윙, 인스타골프, 미즈...",[mizuno_golfapparel_kr],193,unknown,2021-08-05T12:33:32.000Z
2,soyoung_pro,593,20천,168,안소영프로\n골프 강사\n🙋🏻‍♀️\n⛳️ KLPGA 507/ TPI ...,soyoung_pro\n⠀\n유띨리띠 🏌🏻‍♀️\n⠀\n@mizuno_golfapp...,"[골스타그램, 골프, 한남동, 트랙맨레슨, 스윙교정, 골프스윙, 골프레슨, 안소영프...",[mizuno_golfapparel_kr],423,unknown,2021-08-04T13:32:19.000Z
3,soyoung_pro,593,20천,168,안소영프로\n골프 강사\n🙋🏻‍♀️\n⛳️ KLPGA 507/ TPI ...,soyoung_pro\n⠀\n@mizuno_golfapparel_kr\n2일,"[골스타그램, 골프, 트랙맨레슨, 한남동, 스윙교정, 골프스윙, 미즈노골프어패럴, ...","[mizuno_golfapparel_kr, lisa_fx_brandon_]",791,unknown,2021-08-03T14:35:06.000Z
4,soyoung_pro,593,20천,168,안소영프로\n골프 강사\n🙋🏻‍♀️\n⛳️ KLPGA 507/ TPI ...,soyoung_pro\n⠀\n드라이버🏌🏻‍♀️\n⠀\n⠀\n@mizuno_golfa...,"[골스타그램, 골프, 트랙맨골프, 골프레슨, 스윙교정, 인스타골프, 한남동, 트랙맨...","[mizuno_golfapparel_kr, LieKay.Official]",428,unknown,2021-08-02T13:29:07.000Z
